# WebScrap the Finance Data, Save it as Excel and Attach the File in Mail

In this yahoo finance has been used to scrap the historical data(past 1 year). Data has been saved in Excel format. Then the file has been shared via gmail. 

### 1. Import the Libs which are required

In [1]:
# import stock_info module from yahoo_fin
# !pip install yfinance 
import yfinance as yf 
import pandas as pd
from datetime import *
import os

#### to send the gmail with attachment 
import smtplib
import getpass
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 


### 2. Get the ticker Data

In [2]:
#define the ticker symbol
tickerSymbol = ["AMZN","MSFT","NFTY","BSE"]

tickerData = []
#get data on this ticker
for data in tickerSymbol:
    #print(data)
    tickerData.append(yf.Ticker(data))
print(tickerData)


[yfinance.Ticker object <AMZN>, yfinance.Ticker object <MSFT>, yfinance.Ticker object <NFTY>, yfinance.Ticker object <BSE>]


### 3. Save data in Excel

In [3]:
# Remove the file if already exist
if os.path.exists('finance.xlsx'):
    os.remove('finance.xlsx')

endDate = date.today();
i = 0

# Write the Data in Excel and save it
writer = pd.ExcelWriter('finance.xlsx', engine = 'xlsxwriter')
for data in tickerData:
    tickerDf = pd.DataFrame()
    tickerDf = data.history(period='1d', start=endDate - timedelta(days=365), end= endDate) # past one year data
    tickerDf.to_excel(writer,sheet_name=tickerSymbol[i])
    i+=1
writer.save()
writer.close()

print("Success! Data saved")

Success! Data saved


In [4]:
tickerDf.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-12-30,13.20,13.24,13.14,13.19,10300,0.0,0
2019-12-31,13.23,13.23,13.18,13.18,5600,0.0,0
2020-01-02,13.21,13.21,13.16,13.18,2200,0.0,0
2020-01-03,13.18,13.23,13.17,13.18,14400,0.0,0
2020-01-06,13.19,13.24,13.18,13.18,7100,0.0,0


### 4. Compose mail, attach the file and mail it

In [ ]:
# Get the from and to address
fromaddrs = input("From: ")
password = getpass.getpass(prompt='Enter the password: ')  
toaddrs = input(prompt="To: ")

# instance of MIMEMultipart 
msg = MIMEMultipart() 
  
# storing the senders email address   
msg['From'] = fromaddrs
  
# storing the receivers email address  
msg['To'] = toaddrs
  
# storing the subject  
msg['Subject'] = "Finance Report for past one year"

# string to store the body of the mail 
body = "Finance Report has been attached."
  
# attach the body with the msg instance 
msg.attach(MIMEText(body, 'plain')) 

# open the file to be sent  
filename = "finance.xlsx"
attachment = open("finance.xlsx", "rb") 
  
# instance of MIMEBase and named as p 
p = MIMEBase('application', 'octet-stream') 
  
# To change the payload into encoded form 
p.set_payload((attachment).read()) 
  
# encode into base64 
encoders.encode_base64(p) 
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  
# attach the instance 'p' to instance 'msg' 
msg.attach(p) 
message = msg.as_string() 

# creates SMTP session 
smtpserver = smtplib.SMTP("smtp.gmail.com", 587)
smtpserver.ehlo()
smtpserver.starttls()
smtpserver.login(fromaddrs, password)
smtpserver.sendmail(fromaddrs, toaddrs, message)  
smtpserver.quit()
print("Mail Sent!!!")